#  **SQL 데이터를 활용한 질문/응답(Q&A) 시스템 구축**

LLM(대규모 언어 모델) 시스템이 **구조화된 데이터(structured data)** 를 쿼리하도록 만드는 것은 **비구조화된 텍스트(unstructured text data)** 를 처리하는 것과는 상당히 다른 접근 방식이 필요합니다.

- **비구조화된 데이터:** 주로 벡터 데이터베이스를 사용하여 텍스트를 검색합니다.  
- **구조화된 데이터:** LLM이 SQL과 같은 언어를 사용하여 **쿼리(query)**를 작성하고 실행합니다.

다음 두 가지 구현 방식을 다룹니다:  
1. **체인(Chains)**
2. **에이전트(Agents)**

이 시스템을 통해 **데이터베이스에 대한 질문을 자연어로 입력**하고, **자연어로 답변을 받을 수 있습니다.**

- **체인:** 단일 SQL 쿼리를 실행하여 답변을 생성합니다.  
- **에이전트:** 필요에 따라 여러 번 데이터베이스를 쿼리하여 답변을 생성합니다.
---

## **아키텍처 (Architecture)**

1. **질문을 SQL 쿼리로 변환:**  
   - 모델이 사용자 입력을 SQL 쿼리로 변환합니다.  
<p></p>
2. **SQL 쿼리 실행:**  
   - 변환된 SQL 쿼리를 데이터베이스에서 실행합니다.  
<p></p>
3. **질문에 답변:**  
   - 모델이 SQL 쿼리 결과를 바탕으로 사용자 질문에 대한 답변을 생성합니다.  

<img src="https://python.langchain.com/assets/images/sql_usecase-d432701261f05ab69b38576093718cf3.png" />

In [ ]:
# !pip install -qU \
# python-dotenv \
# langchain \
# langchain-community \
# openai \
# anthropic \
# langchain-openai \
# langchain-anthropic \
# langchain-google-genai \
# python-dotenv \
# langchainhub \
# langgraph

In [ ]:
# LangChain 추적(Tracing) 설정 활성화

### **샘플 데이터**

이 예제에서는 **SQLite** 연결을 사용하여 **Chinook 데이터베이스**를 활용합니다.  
Chinook 데이터베이스는 **디지털 미디어 스토어**를 나타내는 샘플 데이터베이스입니다.

---

##  **Chinook 데이터베이스 설치**

[설치 가이드](https://database.guide/2-sample-databases-sqlite/)를 따라 `Chinook.db`를 현재 노트북과 같은 디렉터리에 생성합니다.  

또는, 아래 명령어를 사용하여 커맨드 라인에서 데이터베이스를 다운로드하고 빌드할 수 있습니다:

```bash
curl -s https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql | sqlite3 Chinook.db
```

### **설명:**  
1. `curl` 명령어를 사용하여 Chinook 데이터베이스 SQL 스크립트를 다운로드합니다.  
2. `sqlite3`를 사용해 SQL 스크립트를 실행하고 `Chinook.db` 파일을 생성합니다.

---
이 데이터베이스는 **SQLAlchemy 기반의 `SQLDatabase` 클래스**를 사용하여 상호작용할 수 있습니다.

In [ ]:
# SQLite3 설치 in Colab
# !sudo apt-get update
# !sudo apt-get install sqlite3

In [ ]:
# Chinook db 설치 in Colab
# !curl -s https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql | sqlite3 Chinook.db

In [ ]:
# SQLite 데이터베이스 파일 'Chinook.db'에 연결합니다.
# 사용 중인 데이터베이스의 SQL 방언(SQL Dialect)을 출력합니다.
# 데이터베이스에서 사용할 수 있는 테이블 이름들을 출력합니다.

##  **SQL 데이터베이스를 LLM과 연결하기**

##  **체인(Chains)**

**체인(Chains)** 은 예측 가능한 단계들의 조합입니다.  
**LangGraph**에서는 체인을 간단한 **노드들의 순서(sequence of nodes)** 로 표현할 수 있습니다.

주어진 질문에 대해 다음 단계를 수행하는 일련의 단계를 LangGraph를 이용하여 만들어 보겠습니다:  

- 질문을 **SQL 쿼리로 변환**합니다.  
- 쿼리를 **실행**합니다.  
- 쿼리 **결과를 사용해 원래 질문에 답변**합니다.  

---

이 구조에서는 다음과 같은 시나리오를 지원하지 않습니다:  

1. 이 시스템은 모든 사용자 입력에 대해 SQL 쿼리를 실행합니다.  
   - 예: `"hello"`라는 입력에도 SQL 쿼리를 실행합니다.  
2. 일부 질문은 **여러 개의 쿼리**가 필요할 수 있습니다.  

이러한 시나리오는 **에이전트(Agents)** 를 이용하여 구현해야 합니다.

---

## **애플리케이션 상태(Application State)**

LangGraph의 **상태(State)** 는 애플리케이션에 입력되는 데이터, 단계 간에 전달되는 데이터, 그리고 출력되는 데이터를 제어합니다.  

- 일반적으로 `TypedDict`로 정의됩니다.  
- 또는, [**Pydantic BaseModel**](https://langchain-ai.github.io/langgraph/how-tos/state-model/)을 사용할 수도 있습니다.

---

###  **이 애플리케이션의 상태(State)**

이 애플리케이션에서는 다음 데이터를 추적합니다:  

1. **입력된 질문 (Input Question)**  
2. **생성된 SQL 쿼리 (Generated Query)**  
3. **쿼리 결과 (Query Result)**  
4. **생성된 답변 (Generated Answer)**  

In [ ]:
# 상태(State) 정의
# - 애플리케이션에서 상태(State)는 단계 간에 전달되는 데이터를 추적하기 위해 사용됩니다.
# - TypedDict를 사용하여 상태에서 사용할 키(key)와 해당 값(value)의 데이터 타입을 정의합니다.
class State(TypedDict):

이제 이 상태(state)를 기반으로 작동하며 내용을 채워주는 **함수**들이 필요합니다.

---

## **질문을 SQL 쿼리로 변환하기**

첫 번째 단계는 **사용자 입력을 SQL 쿼리로 변환**하는 것입니다.  
SQL 쿼리를 신뢰성 있게 생성하기 위해(마크다운 형식, 설명 또는 부가적인 내용 없이) LangChain의 **구조화된 출력(structured output)** 추상화를 사용할 것입니다.

In [ ]:
# from langchain_anthropic import ChatAnthropic
# llm = ChatAnthropic(model="claude-3-5-haiku-20241022")

---------
이 프롬프트에는 **SQL 방언(dialect)** 과 **테이블 정보(table_info)** 와 같은 채워야 할 여러 매개변수들이 포함되어 있습니다.  

LangChain의 [`SQLDatabase`](https://python.langchain.com/api_reference/community/utilities/langchain_community.utilities.sql_database.SQLDatabase.html) 객체에는 이러한 작업을 도와주는 메서드들이 포함되어 있습니다.  

우리의 `write_query` 단계는 이 매개변수들을 채운 뒤, **모델을 프롬프트하여 SQL 쿼리를 생성**하도록 할 것입니다.

In [ ]:
# SQL 쿼리 프롬프트 템플릿 정의
class QueryOutput(TypedDict):
def write_query(state: State):
# 테스트

### **쿼리 실행 (Execute query)**

**이 단계는 SQL 체인을 만드는 과정에서 가장 주의할 부분입니다.**  
자동화된 쿼리를 데이터베이스에서 실행해도 안전한지 신중하게 검토해야 합니다.  
**데이터베이스 연결 권한은 가능한 한 최소화**하세요.  
또한, 쿼리를 실행하기 전에 **사람의 승인 단계(human approval step)** 를 체인에 추가하는 것을 고려하세요.

In [ ]:
def execute_query(state: State):
    # QuerySQLDatabaseTool을 사용하여 데이터베이스에 쿼리 실행
    # state["query"]에 저장된 SQL 쿼리를 실행하고 결과 반환
# SQL 쿼리 실행 예제

###  **답변 생성 (Generate answer)**  

마지막 단계에서는 **데이터베이스에서 가져온 정보**를 바탕으로 질문에 대한 **답변을 생성**합니다.

In [ ]:
def generate_answer(state: State):

### **LangGraph를 사용한 오케스트레이션 (Orchestrating with LangGraph)**  

마지막으로, 우리의 애플리케이션을 하나의 **`graph` 객체**로 컴파일합니다.  
이 경우, **세 가지 단계를 순서(sequence)** 로 연결하기만 하면 됩니다.

In [ ]:
# StateGraph 생성
# 그래프의 시작점(START)과 첫 번째 단계(write_query) 연결
# 정의된 단계와 연결 관계를 기반으로 최종 그래프 생성

각 단계를 개별적으로 **스트리밍(stream)** 하여 결과를 확인할 수 있습니다.

[LangSmith trace](https://smith.langchain.com/o/351c6cd9-1396-5c74-9478-1ee6a22a6433/projects/p/acec9d4d-4978-4597-adff-789cd42e200f?timeModel=%7B%22duration%22%3A%227d%22%7D&peek=2f11a326-ed09-4c35-b057-211164093e1a)를 check해 봅니다.

###  **Human-in-the-loop (인간 개입 단계)**  

**LangGraph**는 이 워크플로우에 유용한 여러 기능을 지원합니다.  
그 중 하나는 **[human-in-the-loop](https://langchain-ai.github.io/langgraph/concepts/human_in_the_loop/)** 입니다.  

- 이 기능을 사용하면 **민감한 단계**(예: SQL 쿼리 실행) 전에 애플리케이션을 **중단(interrupt)** 하고 **사람이 검토(human review)** 할 수 있습니다.  

- 이 기능은 LangGraph의 **[persistence(영속성)](https://langchain-ai.github.io/langgraph/concepts/persistence/)** 레이어를 통해 지원됩니다.  
- 이 레이어는 애플리케이션의 실행 진행 상태(run progress)를 사용자가 선택한 저장소(storage)에 저장합니다.  

아래 예시에서는 **인메모리(in-memory)** 저장소를 지정합니다.

In [ ]:
# 애플리케이션의 실행 상태(run progress)를 인메모리(in-memory) 저장소에 저장
# 이를 통해 실행이 중단되거나 검토가 필요한 경우, 저장된 상태에서 다시 시작할 수 있습니다.
# 그래프 컴파일 및 중단(interrupt) 지점 설정
# - `checkpointer`: MemorySaver를 사용하여 실행 상태를 저장합니다.
# - `interrupt_before`: "execute_query" 단계 전에 실행을 중단하여 사람이 검토(human review)할 수 있도록 합니다.
# 영속성을 사용할 때는 실행을 이어갈 수 있도록 고유 식별 스레드 ID를 지정

----------
동일한 실행(run)을 반복하되, 간단한 **예/아니오(yes/no) 승인 단계**를 추가해보겠습니다.

In [ ]:
# 그래프 실행 및 스트림 모드로 단계별 결과 확인
# execute_query 단계 전에 사용자 승인 여부 확인
# 승인 확인
    # 사용자가 'no'를 입력한 경우, 실행 중단

##  **에이전트(Agents)**

**에이전트(Agents)**는 **LLM의 추론(reasoning) 능력**을 활용하여 실행 중에 결정을 내립니다.  
에이전트를 사용하면 **쿼리 생성 및 실행 과정의 추가적인 판단(discretion)** 을 LLM에 맡길 수 있습니다.  

**체인(Chain)** 보다 예측 가능성이 떨어질 수 있지만, 에이전트에는 몇 가지 장점이 있습니다:

- 사용자 질문에 답하기 위해 **필요한 만큼 데이터베이스를 여러 번 쿼리**할 수 있습니다.  
- 생성된 쿼리를 실행하다 오류가 발생하면, **오류(traceback)를 포착**하여 올바르게 **재생성(regenerate)** 할 수 있습니다.  
- 데이터베이스의 **스키마(schema)** 와 **내용(content)** 모두를 기반으로 질문에 답할 수 있습니다 (예: 특정 테이블 설명).  

---

### **SQL 에이전트 구성하기**

아래에서는 최소한의 **SQL 에이전트(SQL Agent)** 를 구성합니다.  
이를 위해 LangChain의 [**SQLDatabaseToolkit**](https://python.langchain.com/api_reference/community/agent_toolkits/langchain_community.agent_toolkits.sql.toolkit.SQLDatabaseToolkit.html)을 사용하여 도구(tool) 세트를 제공합니다.  

**LangGraph의 [사전 구축된 ReAct 에이전트 생성자(pre-built ReAct agent constructor)](https://langchain-ai.github.io/langgraph/how-tos/#langgraph.prebuilt.chat_agent_executor.create_react_agent)** 를 사용하면 **한 줄의 코드**로 이를 수행할 수 있습니다.

---

`SQLDatabaseToolkit`에는 다음과 같은 도구들이 포함됩니다:

- **쿼리 생성 및 실행(Create and execute queries)**  
- **쿼리 구문 검증(Check query syntax)**  
- **테이블 설명 가져오기(Retrieve table descriptions)**  
- **기타 여러 기능(... and more)**  

In [ ]:
# SQLDatabaseToolkit은 LLM과 데이터베이스(SQL)를 연결하기 위한 도구 세트
# SQLDatabaseToolkit에서 사용할 수 있는 모든 도구를 가져옵니다.
# 도구 목록 출력

### **시스템 프롬프트 (System Prompt)**  

에이전트가 올바르게 동작하도록 하기 위해 **시스템 프롬프트(System Prompt)** 를 로드할 필요가 있습니다.   
이를 위해 [**Prompt Hub**](https://smith.langchain.com/hub)에서 sql 생성 **시스템 프롬프트** 를 가져올 것입니다.  
이 프롬프트에는 **에이전트의 행동 지침(instructions for how to behave)** 이 포함됩니다.

In [ ]:
#  SQL 에이전트를 위한 시스템 프롬프트 템플릿
# 이 템플릿에는 SQL 에이전트가 어떻게 행동해야 하는지에 대한 지침이 포함되어 있습니다.
# prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")
# prompt_template = hub.pull("oyj-1/sql-agent-system-prompt-korean")
# 프롬프트 내용 출력

프롬프트에서 강조된 **매개변수(parameters)** 를 채워보겠습니다.

In [ ]:
#system_message

### **에이전트 초기화 (Initializing Agent)**  

사전 구축된 **LangGraph** 에이전트를 사용하여 **에이전트(agent)** 를 생성합니다.

In [ ]:
# ReAct(Reasoning + Acting) 패턴을 사용하여 에이전트 생성

In [ ]:
# SQL 쿼리로 검증

[**LangSmith 추적(Trace)**](https://smith.langchain.com/o/351c6cd9-1396-5c74-9478-1ee6a22a6433/projects/p/acec9d4d-4978-4597-adff-789cd42e200f?timeModel=%7B%22duration%22%3A%227d%22%7D&peek=b67c9ebf-e953-4d79-9b34-c0dbbeb921c4)을 사용하여 이러한 단계와 관련 메타데이터를 시각화할 수도 있습니다.

에이전트는 필요한 정보를 얻을 때까지 **여러 번의 쿼리**를 실행한다는 점에 주목하세요:  

1. 사용 가능한 테이블 목록을 나열합니다.  
2. 세 개의 테이블에 대한 스키마를 검색합니다.  
3. 여러 테이블을 **JOIN 연산**을 통해 쿼리합니다.  

에이전트는 마지막 쿼리의 결과를 사용해 **원래 질문에 대한 답변을 생성**할 수 있습니다.

에이전트는 이와 유사하게 **정성적 질문(qualitative questions)** 도 처리할 수 있습니다.